In [1]:
!pip install Faker
!pip install boto3
!pip install awswrangler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 7.8 MB/s eta 0:00:00:00:01


# Desafio #1. Generando de forma automática los datos de: departamentos, puestos de trabajo, y empleados

In [4]:
from faker import Faker
import random
import pandas as pd
import boto3
#import fastparquet as fp
from io import BytesIO
import awswrangler as wr

# Semilla
random.seed(42)

# Configura el generador Faker con idioma en español
fake = Faker('es_ES')

# Nombres de departamentos ficticios de Colombia
departamentos_colombia = ['Amazonas', 'Antioquia', 'Arauca', 'Atlántico', 'Bolívar', 'Boyacá', 'Caldas', 'Caquetá', 'Casanare', 'Cauca', 'Cesar', 'Chocó', 'Córdoba', 'Cundinamarca', 'Guainía', 'Guaviare', 'Huila', 'La Guajira', 'Magdalena', 'Meta', 'Nariño', 'Norte de Santander', 'Putumayo', 'Quindío', 'Risaralda', 'San Andrés y Providencia', 'Santander', 'Sucre', 'Tolima', 'Valle del Cauca', 'Vaupés', 'Vichada']

# Nombres de puestos de trabajo ficticios
puestos_trabajo = ['Gerente', 'Asistente', 'Analista', 'Desarrollador', 'Diseñador', 'Contador', 'Ingeniero', 'Vendedor', 'Recepcionista', 'Especialista', 'Coordinador', 'Supervisor', 'Director']

# Genera datos de empleados ficticios
def generar_empleado():
    nombre = fake.name()
    departamento = random.choice(departamentos_colombia)
    puesto = random.choice(puestos_trabajo)
    salario = random.randint(2000000, 10000000)  # Salario aleatorio entre $200,0000 y $10,000,000
    email = fake.email()
    telefono = fake.phone_number()
    direccion = fake.address()
    return {
        'Nombre': nombre,
        'Departamento': departamento,
        'Puesto de trabajo': puesto,
        'Salario': salario,
        'Email': email,
        'Telefono': telefono,
        'Direccion': direccion
    }

# Genera una lista de empleados ficticios
def generar_empleados(numero_empleados):
    """
    Input:
    numero_empleados -> Numero de empleados que se desea generar
    
    Output:
    return -> Lista de diccionarios con el numero de empleados generados con los siguientes atributos:
              Nombre, Departamento, Puesto de trabajo, Salario, Email, Teléfono', Dirección
    """
    empleados = []
    for _ in range(numero_empleados):
        empleados.append(generar_empleado())
    return empleados

# Generar 10000 empleados ficticios
empleados_generados = pd.DataFrame(generar_empleados(10000))
empleados_generados.to_csv("empleados.csv")

# Desafio #2. Guardando los datos simulados en archivos con formato Parquet

In [10]:
# Creando el cliente de S3
s3 = boto3.client('s3')

# Nombre del archivo y el nombre del bucket de S3
nombre_archivo = 'empleados.parquet'
nombre_bucket = 'luis-ds-db'

# Convirtiendo el DataFrame a formato Parquet en memoria
parquet_buffer = BytesIO()
wr.s3.to_parquet(df=empleados_generados, path=f's3://{nombre_bucket}/{nombre_archivo}', index=False)

print("Los datos de empleados han sido guardados exitosamente en el bucket")

Los datos de empleados han sido guardados exitosamente en el bucket
